In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np


In [190]:
from data_master_eng_ml.config import (
    GAME_INFO_RAW_COLLECTION,
    MULTIPLAYER_MODES_RAW_COLLECTION,
    INVOLVED_COMPANIES_LIST_RAW_COLLECTION,
    INVOLVED_COMPANIES_RAW_COLLECTION,
    GAME_RELEASE_DATES_RAW_COLLECTION,
    MONGODB_DATABASE_RAW
)

In [3]:
ano = "2022"

In [171]:
from data_master_eng_ml.transformations.api_transformations import (
    batch_fetch_age_classifications
)
from data_master_eng_ml.utils.helpers import map_age_classifications,generate_dummies,read_data_from_mongodb


In [238]:
from data_master_eng_ml.dataset import main as main_raw

In [239]:
from data_master_eng_ml.features import main as main_silver

In [191]:
single = True
year = 2022
game_id = 311813

In [241]:
main_raw(year=year)

2024-09-07 22:07:18.272 | INFO     | data_master_eng_ml.dataset:main:134 - Starting data ingestion for the year: 2022
2024-09-07 22:07:18.274 | INFO     | data_master_eng_ml.dataset:ingest_raw_data:50 - Ingesting raw data for the year: 2022
2024-09-07 22:07:18.274 | DEBUG    | data_master_eng_ml.dataset:ingest_raw_data:53 - Fetching game release dates batch...
2024-09-07 22:07:18.274 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_batch:82 - Constructing the query and initiating data fetch...
2024-09-07 22:07:23.162 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_batch:85 - Data fetched successfully.
2024-09-07 22:07:23.162 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_batch:87 - Returning the resulting DataFrame.
2024-09-07 22:07:23.166 | DEBUG    | data_master_eng_ml.dataset:ingest_raw_data:61 - Fetching involved companies batch...
2024-09-07 22:07:23.166 | 

Uploading to collection game_release_dates_raw: 100%|██████████| 4/4 [00:12<00:00,  3.12s/it]


2024-09-07 22:07:45.877 | INFO     | data_master_eng_ml.utils.helpers:save_dataframe_to_mongodb:49 - Data successfully inserted into MongoDB!
2024-09-07 22:07:45.879 | INFO     | data_master_eng_ml.dataset:ingest_raw_data:88 - Saving companies list data to MongoDB...


Uploading to collection involved_companies_list_raw: 100%|██████████| 2/2 [00:05<00:00,  2.98s/it]


2024-09-07 22:07:51.852 | INFO     | data_master_eng_ml.utils.helpers:save_dataframe_to_mongodb:49 - Data successfully inserted into MongoDB!
2024-09-07 22:07:51.854 | INFO     | data_master_eng_ml.dataset:ingest_raw_data:95 - Saving companies data to MongoDB...


...

2024-09-07 22:07:56.848 | INFO     | data_master_eng_ml.utils.helpers:save_dataframe_to_mongodb:49 - Data successfully inserted into MongoDB!
2024-09-07 22:07:56.853 | INFO     | data_master_eng_ml.dataset:ingest_raw_data:102 - Saving multiplayer modes to MongoDB...


Uploading to collection multiplayer_modes_raw: 100%|██████████| 1/1 [00:01<00:00,  1.01s/it]


2024-09-07 22:07:57.875 | INFO     | data_master_eng_ml.utils.helpers:save_dataframe_to_mongodb:49 - Data successfully inserted into MongoDB!
2024-09-07 22:07:57.875 | INFO     | data_master_eng_ml.dataset:ingest_raw_data:111 - Saving game information to MongoDB...


Uploading to collection game_info_raw: 100%|██████████| 3/3 [00:06<00:00,  2.20s/it]

2024-09-07 22:08:04.504 | INFO     | data_master_eng_ml.utils.helpers:save_dataframe_to_mongodb:49 - Data successfully inserted into MongoDB!
2024-09-07 22:08:04.506 | INFO     | data_master_eng_ml.dataset:ingest_raw_data:117 - Data ingestion completed successfully.
2024-09-07 22:08:04.509 | SUCCESS  | data_master_eng_ml.dataset:main:140 - Processing dataset complete.


In [242]:
main_silver(year=year)

2024-09-07 22:08:04.538 | INFO     | data_master_eng_ml.features:main:122 - Generating features from dataset...
2024-09-07 22:08:04.538 | INFO     | data_master_eng_ml.features:featurization:45 - Starting data featurization for the year: 2022
2024-09-07 22:08:04.538 | DEBUG    | data_master_eng_ml.features:featurization:51 - Reading multiplayer modes data from MongoDB...
2024-09-07 22:08:05.773 | DEBUG    | data_master_eng_ml.features:featurization:56 - Reading involved companies list data from MongoDB...
2024-09-07 22:08:07.826 | DEBUG    | data_master_eng_ml.features:featurization:61 - Reading game release dates data from MongoDB...
2024-09-07 22:08:09.893 | DEBUG    | data_master_eng_ml.features:featurization:67 - Extracting features from game release dates data...
2024-09-07 22:08:09.894 | DEBUG    | data_master_eng_ml.transformations.api_feature_data:fetch_game_release_dates_feature:34 - Mapping region codes to region names...
2024-09-07 22:08:09.896 | INFO     | data_master_eng_m

Uploading to collection cleaned_games_info: 100%|██████████| 2/2 [00:03<00:00,  1.76s/it]

2024-09-07 22:08:19.611 | INFO     | data_master_eng_ml.utils.helpers:save_dataframe_to_mongodb:49 - Data successfully inserted into MongoDB!
2024-09-07 22:08:19.613 | INFO     | data_master_eng_ml.features:featurization:108 - Final DataFrame saved to MongoDB successfully.
2024-09-07 22:08:19.618 | SUCCESS  | data_master_eng_ml.features:main:127 - Features generation complete.


In [198]:
query = {"dat_ref": year}

In [121]:
data_frame_games_find_raw_single = fetch_raw_game_release_dates_single(game_id=game_id)

2024-09-07 19:33:57.374 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_single:41 - Fetching release dates for game ID: 311813
2024-09-07 19:33:57.377 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_single:58 - Constructing the query and initiating data fetch...
2024-09-07 19:33:57.899 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_single:61 - Data fetched successfully.
2024-09-07 19:33:57.899 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_single:63 - Returning the resulting DataFrame.


In [130]:
games_id_batch = list(set(data_frame_games_find_raw_batch["game"].to_list()))

In [131]:
games_id_list_batch = ",".join(map(str, games_id_batch))

In [125]:
games_id_single = list(set(data_frame_games_find_raw_single["game"].to_list()))

In [126]:
games_id_list_single = ",".join(map(str, games_id_single))

In [137]:
data_frame_companies_find_raw_single = fetch_raw_involved_companies(games_id_list_single)
data_frame_companies_find_raw_single
company_id_list_single = list(set(data_frame_companies_find_raw_single["company"].to_list()))


2024-09-07 19:41:25.416 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_involved_companies:121 - Fetching involved companies for game ID:
2024-09-07 19:41:25.416 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_involved_companies:127 - Constructing the query and initiating data fetch...
2024-09-07 19:41:26.316 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_involved_companies:130 - Data fetched successfully.
2024-09-07 19:41:26.317 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_involved_companies:132 - Returning the resulting DataFrame.


In [146]:
data_frame_multiplayer_modes_raw_single = fetch_raw_multiplayer_modes(games_id_list_single)
data_frame_multiplayer_modes_raw_single

2024-09-07 19:48:39.101 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_multiplayer_modes:194 - Fetching multiplayer modes for game ID: 311813
2024-09-07 19:48:39.102 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_multiplayer_modes:211 - Constructing the query and initiating data fetch...
2024-09-07 19:48:39.477 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_multiplayer_modes:214 - Data fetched successfully.
2024-09-07 19:48:39.477 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_multiplayer_modes:217 - Ensuring all expected columns are present in the DataFrame.
2024-09-07 19:48:39.480 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_multiplayer_modes:220 - Returning the processed DataFrame.


,campaigncoop,game,lancoop,offlinecoop,offlinecoopmax,offlinemax,onlinecoop,onlinecoopmax,onlinemax,splitscreen


In [57]:
data_frame_games = fetch_game_info(games_id_list)